In [1]:
from sympy import *
import math
import numpy as np

### 19.1

In [2]:
E=20e6

In [3]:
def PlaneTrussElementLength(x1,y1,x2,y2):
    return math.sqrt((x2-x1)*(x2-x1) + (y2-y1)*(y2-y1))

In [4]:
def PlaneTrussElementStiffness(E,A,L,theta):
    x = theta*math.pi/180
    C = math.cos(x)
    S = math.sin(x)
    return E*A/L*np.array([[C*C, C*S, -C*C, -C*S],[C*S, S*S, -C*S, -S*S],
                           [-C*C, -C*S, C*C, C*S],[-C*S, -S*S, C*S, S*S]]) 

In [5]:
def PlaneTrussAssemble(K,k,i,j):
    # K - np.array
    # k - np.array
    K[2*i,2*i] = K[2*i,2*i] + k[0,0]
    K[2*i,2*i+1] = K[2*i,2*i+1] + k[0,1] 
    K[2*i,2*j] = K[2*i,2*j] + k[0,2] 
    K[2*i,2*j+1] = K[2*i,2*j+1] + k[0,3]
    K[2*i+1,2*i] = K[2*i+1,2*i] + k[1,0]
    K[2*i+1,2*i+1] = K[2*i+1,2*i+1] + k[1,1] 
    K[2*i+1,2*j] = K[2*i+1,2*j] + k[1,2] 
    K[2*i+1,2*j+1] = K[2*i+1,2*j+1] + k[1,3] 
    K[2*j,2*i] = K[2*j,2*i] + k[2,0] 
    K[2*j,2*i+1] = K[2*j,2*i+1] + k[2,1]
    K[2*j,2*j] = K[2*j,2*j] + k[2,2]
    K[2*j,2*j+1] = K[2*j,2*j+1] + k[2,3] 
    K[2*j+1,2*i] = K[2*j+1,2*i] + k[3,0] 
    K[2*j+1,2*i+1] = K[2*j+1,2*i+1] + k[3,1]
    K[2*j+1,2*j] = K[2*j+1,2*j] + k[3,2] 
    K[2*j+1,2*j+1] = K[2*j+1,2*j+1] + k[3,3]
    
    return K  

In [6]:
def FEA_u(coord, elcon, bc_node, bc_val, global_force,  A, E):
    K=np.zeros(shape=(2*np.max(elcon)+2,2*np.max(elcon)+2))
    i=0
    for el in elcon:
        L=PlaneTrussElementLength(coord[el[0]][0],coord[el[0]][1],coord[el[1]][0],coord[el[1]][1])
        theta=math.atan((coord[el[1]][1]-coord[el[0]][1])/(coord[el[1]][0]-coord[el[0]][0]+1e-13))*180/math.pi
        k=PlaneTrussElementStiffness(E, A[i], L, theta)
        K=PlaneTrussAssemble(K,k,el[0],el[1])
        i+=1
    
    F = global_force
    
    bc=np.array([bc_node, 
                bc_val]).T
    nd, nd=K.shape
    fdof=np.array([i for i in range(nd)]).T
    d=np.zeros(shape=(len(fdof),))
    Q=np.zeros(shape=(len(fdof),))

    pdof=bc[:,0].astype(int)
    dp=bc[:,1]
    fdof=np.delete(fdof, pdof, 0)
    s=np.linalg.lstsq(K[fdof,:][:,fdof],
                      (F[fdof].T-np.dot(K[fdof,:][:,pdof],dp.T)).T, rcond=None)[0] 
    d[pdof]=dp
    d[fdof]=s.reshape(-1,)
    
#     Q=np.dot(K,d).T-F 
    
    return d

In [7]:
coord=np.array([0,0,
               0,-80,
               60,-80]).reshape(3,2)

In [8]:
elcon=np.array([[0, 2],
                [1, 2]])

In [9]:
bc_node = [0, 1, 2, 3]

In [10]:
bc_val=[0., 0., 0., 0.]

In [11]:
theta3=30

In [12]:
x3 = theta3*math.pi/180
C3 = math.cos(x3)
S3 = math.sin(x3)

In [13]:
global_force=np.array([0,0,
                       0,0,
                       50000*S3, -50000*C3])

In [14]:
A=[5,12]

In [15]:
FEA_u(coord, elcon, bc_node, bc_val, global_force,  A, E)

array([ 0.        ,  0.        ,  0.        ,  0.        , -0.00186899,
       -0.06905998])

### 19.3

In [16]:
coord=np.array([0,0,
               400,400,
               800,0]).reshape(3,2)

In [17]:
elcon=np.array([[0, 1],
                [0,2],
                [1, 2]])

In [18]:
bc_node = [0, 1, 5]

In [19]:
bc_val=[0., 0., 0.]

In [20]:
global_force=np.array([0,0,
                       0,-150e3,
                       0,0])

In [21]:
A=[20,15,20]

In [22]:
FEA_u(coord, elcon, bc_node, bc_val, global_force,  A, E)

array([ 0.        ,  0.        ,  0.1       , -0.31213203,  0.2       ,
        0.        ])

### 19.5

In [23]:
coord=np.array([0,0,
               0,200,
               -100,200,
               200,200]).reshape(4,2)

In [24]:
elcon=np.array([[0, 2],
                [0, 1],
                [0, 3]])

In [25]:
bc_node = [2,3,4,5,6,7]

In [26]:
bc_val=[0., 0., 0., 0., 0., 0.]

In [27]:
global_force=np.array([0,-1000,
                       0,0,
                       0,0,
                       0,0])

In [28]:
A=[8,10,8]

In [29]:
FEA_u(coord, elcon, bc_node, bc_val, global_force,  A, E)

array([-4.26954517e-06, -5.39011092e-04,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

### 19.7

In [56]:
def PlaneTrussElementForce(E,A,L,theta,u):
    x = theta* math.pi/180
    C = math.cos(x)
    S = math.sin(x)
    return E*A/L*np.dot(np.array([-C, -S, C, S]),u)

In [57]:
U1=0
U2=0
U3=1.748
U4=-0.3333
U5=-0.5
U6=0
U7=2.248
U8=-3.69

In [58]:
u=[U1,U2,U3,U4,U5,U6,U7,U8]

In [59]:
coord=np.array([0,0,
               500,500,
               500,0,
               1000,500]).reshape(4,2)

In [60]:
elcon=np.array([[0, 1],
#                 [0, 2],
                [1, 2],
                [1, 3],
#                 [2, 3]
               ])

In [61]:
A=[10,
#    10,
   15,
   10,
#    15
  ]

In [62]:
i=0
for el in elcon:
    L=PlaneTrussElementLength(coord[el[0]][0],coord[el[0]][1],coord[el[1]][0],coord[el[1]][1])
    theta=math.atan((coord[el[1]][1]-coord[el[0]][1])/(coord[el[1]][0]-coord[el[0]][0]+1e-13))*180/math.pi
    print(PlaneTrussElementForce(E,A[i],L,theta,np.array([u[2*el[0]],u[2*el[0]+1],u[2*el[1]],u[2*el[1]+1]])))
    i+=1

282940.0
-199980.00000000038
200000.0000000001


### 19.10

In [63]:
U1=-0.3333
U2=-2.780
U3=0.87
U4=-1.847
U5=-0.2667
U6=-0.9590
U7=0.7812
U8=0
U9=0
U10=0

In [64]:
u=[U1,U2,U3,U4,U5,U6,U7,U8,U9,U10]

In [65]:
coord=np.array([0,0,
               200,-300,
               400,0,
               600,-300,
               800,0]).reshape(5,2)

In [66]:
elcon=np.array([
                [0, 1],
                [0, 2],
#                 [1, 2],
#                 [1, 3],
#                 [2, 3],
#                 [2, 4],
#                 [3, 4]
               ])

In [67]:
A=[10,10,
#    10,15,10,10,10
  ]

In [96]:
i=0
for el in elcon:
    L=PlaneTrussElementLength(coord[el[0]][0],coord[el[0]][1],coord[el[1]][0],coord[el[1]][1])
    theta=math.atan((coord[el[1]][1]-coord[el[0]][1])/(coord[el[1]][0]-coord[el[0]][0]+1e-13))*180/math.pi
    print(PlaneTrussElementForce(E,A[i],L,theta,np.array([u[2*el[0]],u[2*el[0]+1],u[2*el[1]],u[2*el[1]+1]])))
    i+=1

-60369.230769230664
33299.99999999999
